# Hecho servicios acumulados

In [ ]:
from datetime import date
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
import pandas as pd
import numpy as np
import yaml

pd.set_option('future.no_silent_downcasting', True)

### Conexión con la base de datos

In [167]:
# Cargar configuraciones
with open('../config.yml', 'r') as f:
  config = yaml.safe_load(f)
  config_oltp = config['fuente']
  config_olap = config['bodega']

# Crear URLs de conexión
url_oltp = (f"{config_oltp['drivername']}://{config_oltp['user']}:{config_oltp['password']}@{config_oltp['host']}:"f"{config_oltp['port']}/{config_oltp['dbname']}")
url_olap = (f"{config_olap['drivername']}://{config_olap['user']}:{config_olap['password']}@{config_olap['host']}:"f"{config_olap['port']}/{config_olap['dbname']}")

# Crear las conexiones
oltp_conn = create_engine(url_oltp)
olap_conn = create_engine(url_olap)

# Función para probar la conexión con las bases de datos
def comprobar_conexionBD(db_engine, db_name):
  try:
    with db_engine.connect() as connection:
      if connection.execute(text("SELECT 1")).fetchone() is not None:
        print(f"Conexión exitosa a base de datos {db_name}")
      else:
        print(f"No se pudo conectar a {db_name}")
  except SQLAlchemyError as e:
    print(f"Error al conectar a base de datos {db_name}: {e}")

comprobar_conexionBD(oltp_conn, config_oltp['dbname'])
comprobar_conexionBD(olap_conn, config_olap['dbname'])


Conexión exitosa a base de datos mensajeriaOLTP
Conexión exitosa a base de datos mensajeriaOLAP


### Módulo de extracción

In [201]:
table_servicio = pd.read_sql_table('mensajeria_servicio', oltp_conn)
table_estadosservicio = pd.read_sql_table('mensajeria_estadosservicio', oltp_conn)
#table_estados = pd.read_sql_table('mensajeria_estado', oltp_conn) # tabla de estados - sin uso aún
#table_sede = pd.read_sql_table('sede', oltp_conn) # revisar
table_usuario = pd.read_sql_table('clientes_usuarioaquitoy', oltp_conn)
dimension_fecha = pd.read_sql('dim_fecha', olap_conn)
dimension_fecha['hora'] = dimension_fecha['hora'].astype(str).str[:5]
hecho_servicios = (table_servicio[['id', 'mensajero_id', 'mensajero2_id', 'mensajero3_id', 'usuario_id']].rename(columns={'mensajero_id' : 'mensajero1_id'}).astype('Int32').replace({pd.NA: None}))
hecho_servicios = (pd.merge(hecho_servicios, table_usuario[['id', 'cliente_id', 'sede_id']].rename(columns={'id' : 'usuario_id'}), 
  how='left', on='usuario_id'))

estados = [
  (1, 'id_fecha_iniciado'),
  (2, 'id_fecha_mensajero'),
  (3, 'id_fecha_recogida'),
  (4, 'id_fecha_entrega'),
  (5, 'id_fecha_terminado')]

hecho_servicios.head()

,id,mensajero1_id,mensajero2_id,mensajero3_id,usuario_id,cliente_id,sede_id
0,34,None,None,None,10,5,6
1,35,7,None,None,8,5,5
2,36,None,None,None,8,5,5
3,41,None,None,None,173,2,2
4,42,None,None,None,173,2,2


### Módulo de transformación

In [ ]:
# mensajero que más servicios prestó de forma global

# Seleccionar el último mensajero que prestó el servicio
hecho_servicios['mensajero_id'] = (hecho_servicios[['mensajero3_id', 'mensajero2_id', 'mensajero1_id']]
  .astype('Int32')
  .replace({pd.NA: None})
  .bfill(axis=1)
  .infer_objects(copy=False)  # Aplicar infer_objects para evitar FutureWarning
  .iloc[:, 0])

hecho_servicios.drop(columns=['mensajero3_id', 'mensajero2_id', 'mensajero1_id'], inplace=True)

# Cuenta las apariciones de cada mensajero
servicios_por_mensajero = hecho_servicios['mensajero_id'].value_counts().reset_index()
servicios_por_mensajero.columns = ['mensajero_id', 'numero_servicios']

# Ordenar de mayor a menor
servicios_por_mensajero = servicios_por_mensajero.astype('Int32').sort_values(by='numero_servicios', ascending=False)

# Mostrar el resultado
# hecho_servicios.sort_values(by='id').head()
servicios_por_mensajero.head()

,mensajero_id,numero_servicios
0,30,2419
1,29,1527
2,15,1522
3,25,1420
4,31,1346


In [ ]:
# número de servicios por cada sede de cada cliente
servicios_por_sede = (hecho_servicios.groupby(['usuario_id', 'cliente_id', 'sede_id'])['id']
  .count()
  .reset_index()
  .rename(columns={'id': 'num_servicios'})
  # Ordena primero por 'cliente_id' ascendente, luego por 'num_servicios' descendente
  .sort_values(by=['cliente_id', 'num_servicios'], ascending=[True, False]))

servicios_por_sede.drop(columns=['usuario_id']).head

<bound method NDFrame.head of      cliente_id  sede_id  num_servicios
83            2        2            104
21            3       38            205
11            4       11             83
12            4       10             30
13            4       14              3
..          ...      ...            ...
97           25       45            295
99           25       45            205
100          25       44             31
98           25       44             25
137          27       85              1

[150 rows x 3 columns]>

In [204]:
for estado_id, nombre in estados: # genera todas las llaves con la dimensión fecha 
  filtered_estados = (table_estadosservicio[['estado_id', 'servicio_id', 'fecha', 'hora']]
    .query(f"estado_id == {estado_id}")
    .drop(columns=['estado_id'])
    .drop_duplicates(subset=['servicio_id']))
  
  filtered_estados['hora'] = filtered_estados['hora'].astype(str).str[:5]

  filtered_estados = (pd.merge(filtered_estados, dimension_fecha[['fecha', 'hora', 'id']],
    how='left', 
    on=['fecha', 'hora'])
    .rename(columns={'id': nombre})
    .drop(columns=['fecha', 'hora']))

  hecho_servicios = (hecho_servicios
    .merge(filtered_estados, how='left', left_on='id', right_on='servicio_id')
    .drop(columns=['servicio_id'])
    .astype('Int32')
    .replace({pd.NA: None}))

for i in range(len(estados) - 1):
  estado_actual = estados[i][1]
  estado_siguiente = estados[i + 1][1]
    
  tiempos_estados = hecho_servicios[['id', estado_actual, estado_siguiente]].rename(columns={'id': 'id_servicio'})
    
  tiempos_estados = (pd.merge(tiempos_estados, dimension_fecha[['id', 'fecha_hora']], 
    how='left', left_on=estado_actual, right_on='id')
    .rename(columns={'fecha_hora': estado_actual.replace("id_", "")})
    .drop(columns=['id', estado_actual]))

  tiempos_estados = (pd.merge(tiempos_estados, dimension_fecha[['id', 'fecha_hora']], 
    how='left', left_on=estado_siguiente, right_on='id')
    .rename(columns={'fecha_hora': estado_siguiente.replace("id_", "")})
    .drop(columns=['id', estado_siguiente]))

  # diferencia de tiempo entre los pares de estados
  tiempos_estados['diferencia_tiempo'] = tiempos_estados[estado_siguiente.replace("id_", "")] - tiempos_estados[estado_actual.replace("id_", "")]

  # diferencia en horas y minutos
  tiempos_estados[f'horas{estado_actual.replace("id_fecha", "")}_{estado_siguiente.replace("id_fecha_", "")}'] = ((tiempos_estados['diferencia_tiempo'].dt.total_seconds() / 3600)
    .round()
    .astype('Int32')
    .replace({pd.NA: None}))
    
  tiempos_estados.drop(columns=[estado_actual.replace("id_", ""), estado_siguiente.replace("id_", ""), 'diferencia_tiempo'])

  hecho_servicios = (pd.merge(hecho_servicios, tiempos_estados.drop(columns=[estado_actual.replace("id_", ""), estado_siguiente.replace("id_", ""), 'diferencia_tiempo']),
    how='left', left_on='id', right_on='id_servicio')
    .drop(columns=['id_servicio']))

tiempos_estados = hecho_servicios[['id', estados[0][1], estados[len(estados) - 1][1]]]

tiempos_estados = (pd.merge(tiempos_estados.rename(columns={'id' : 'id_servicio'}), dimension_fecha[['id', 'fecha_hora']], 
  how='inner', left_on=estados[0][1], right_on='id')
  .drop(columns=['id_fecha_iniciado', 'id'])
  .rename(columns={'fecha_hora' : 'fecha_hora_iniciado'}))

tiempos_estados = (pd.merge(tiempos_estados, dimension_fecha[['id', 'fecha_hora']], 
  how='inner', left_on='id_fecha_terminado', right_on='id')
  .drop(columns=['id_fecha_terminado', 'id'])
  .rename(columns={'fecha_hora' : 'fecha_hora_terminado'}))

tiempos_estados['horas_iniciado_terminado'] = (
  ((tiempos_estados['fecha_hora_terminado'] - tiempos_estados['fecha_hora_iniciado']).dt.total_seconds() / 3600)
  .round()
  .astype('Int32'))

hecho_servicios = (pd.merge(hecho_servicios, tiempos_estados.drop(columns=['fecha_hora_iniciado', 'fecha_hora_terminado']), 
  how='left', left_on='id', right_on='id_servicio')
  .drop(columns=['id_servicio'])
  .replace({pd.NA: None}))

#print(hecho_servicios.shape[0])
hecho_servicios.sort_values(by=['id']).head()
#tiempos_estados.head(2)

,id,usuario_id,cliente_id,sede_id,mensajero_id,id_fecha_iniciado,id_fecha_mensajero,id_fecha_recogida,id_fecha_entrega,id_fecha_terminado,horas_iniciado_mensajero,horas_mensajero_recogida,horas_recogida_entrega,horas_entrega_terminado,horas_iniciado_terminado
28324,7,173,2,2,7,301,624,None,642,643,577,None,None,5,1009
16,8,173,2,2,7,302,735,None,4779,23045,2212,None,None,1321,4872
2112,9,173,2,2,7,302,770,None,None,None,2403,None,None,None,None
28386,10,173,2,2,7,304,770,None,5799,12814,2403,None,None,514,4145
28385,11,173,2,2,7,305,711,None,1862,None,1941,None,None,None,None


In [ ]:
# Servicios por mes por cada año + mes de cada año que más servicios tuvo
hecho_servicios_temporal = hecho_servicios[['id', 'id_fecha_iniciado']].rename(columns={'id' : 'id_servicio'})

hecho_servicios_temporal = (pd.merge(hecho_servicios_temporal, dimension_fecha[['id', 'año','mes']].astype('Int32'), how='left', left_on='id_fecha_iniciado', right_on='id').drop(columns=['id', 'id_fecha_iniciado']))

hecho_servicios_temporal = hecho_servicios_temporal.groupby(['año', 'mes'])['id_servicio'].count().reset_index().rename(columns={'id_servicio': 'cantidad_servicios'})

max_servicios_por_año = hecho_servicios_temporal.loc[hecho_servicios_temporal.groupby('año')['cantidad_servicios'].idxmax()]

print(hecho_servicios_temporal)
print(max_servicios_por_año)

     año  mes  cantidad_servicios
0   2023    9                  21
1   2023   10                  12
2   2023   11                  17
3   2023   12                  25
4   2024    1                 296
5   2024    2                2479
6   2024    3                3337
7   2024    4                4480
8   2024    5                4725
9   2024    6                4184
10  2024    7                4549
11  2024    8                4304
    año  mes  cantidad_servicios
3  2023   12                  25
8  2024    5                4725


In [ ]:
# Mes con más servicios en general
hecho_servicios_temporal = hecho_servicios[['id', 'id_fecha_iniciado']].rename(columns={'id' : 'id_servicio'}) 

hecho_servicios_temporal = (pd.merge(hecho_servicios_temporal, dimension_fecha[['id', 'año','mes']].astype('Int32'), how='left', left_on='id_fecha_iniciado', right_on='id').drop(columns=['id', 'id_fecha_iniciado'])) 

hecho_servicios_temporal = hecho_servicios_temporal.groupby('mes')['id_servicio'].count().reset_index() 

hecho_servicios_temporal.rename(columns={'id_servicio': 'cantidad_servicios'}, inplace=True) 

print(hecho_servicios_temporal)

    mes  cantidad_servicios
0     1                 296
1     2                2479
2     3                3337
3     4                4480
4     5                4725
5     6                4184
6     7                4549
7     8                4304
8     9                  21
9    10                  12
10   11                  17
11   12                  25


In [164]:
# Agrupar por cliente_id y contar el número de servicios
conteo_servicios_por_cliente = (hecho_servicios.groupby('cliente_id')['id'].count().reset_index().rename(columns={'id': 'num_servicios'}))

# Mostrar el resultado
#print(conteo_servicios_por_cliente.shape[0])
print(conteo_servicios_por_cliente.head(20))

    cliente_id  num_servicios
0            2            104
1            3            205
2            4            117
3            5           4578
4            6            292
5            7           2290
6            8            296
7            9            656
8           11          17384
9           12           1409
10          22             62
11          24             28
12          25           1008
13          27              1


### Módulo de carga a la bodega de datos

In [38]:
hecho_servicios.set_index('id', inplace=True)
try:
  hecho_servicios.to_sql('hecho_servicios', olap_conn, if_exists='replace')
except Exception as e:
  print(f"Error al cargar datos: {e}")